In [ ]:
def remove_suffix(title):
    suffices=[": novellroman",": en roman",": en berättelse",": detektivroman",": gymnasial berättelse",": en berättelse",": Roman",": dramatisk roman",": kriminalroman",": självbiografisk berättelse",": historisk roman",": historisk äventyrsroman",": en kärleksthriller",": en äventyrsberättelse",": en ungdomsbok",": [roman]" ]
    for x in suffices:
        if x in title:
            title=title.replace(x,"")
            break
    return title

In [ ]:
from kblab import Archive
import requests
from requests.auth import HTTPBasicAuth
from lxml import etree
import xml.etree.ElementTree as ET
import time
romaner={"meta.host_title": "Welfare state analytics"}
blm={"label": "BONNIERS", "tags": "issue"}
afton={"tags": "issue"}

def titles_mapping(filter=romaner,max_number=6000):
    with open('/home/liamtabibzadeh/Documents/API_credentials.txt', 'r') as file:
        pw = file.read().replace('\n', '')
    a = Archive("https://datalab.kb.se", auth=("demo", pw))
    
    books_meta={}
    for package_id in a.search(filter, max=max_number):
        for x in a.get(package_id):
            if "aip.mets.metadata" in x:
                for i in range(5):
                    backoff_time = 0.1 * (2 ** i)
                    meta=requests.get(f"https://datalab.kb.se/{package_id}/{x}", auth=HTTPBasicAuth("demo", pw),stream=True)
                    if meta.status_code == 200:
                        root=ET.fromstring(meta.text)
                        meta_data=root.attrib
                        title=meta_data["LABEL"]
                        title=remove_suffix(title)
                        books_meta[title]=package_id
                
                        break
            
                    else:
                        time.sleep(backoff_time)
                break
    return books_meta

In [ ]:

def get_meta_data(path):
    import pandas as pd
    meta_data=pd.read_excel(path)
    meta_data["Datalab ID"]=""

    for key,value in titles_mapping().items():
        for i in range(len(meta_data)):
            if key==meta_data.iloc[i]["Titel"]:
                meta_data.loc[i,"Datalab ID"]+=value

        if key not in list(meta_data["Titel"]):
            meta_data.loc[len(meta_data)] = [key,0,0,0,0,0,value]
    return meta_data


